# Windows Live Response Collection 

When responding to a possible incident you maybe need to collect data to further determine next steps for your analysis  steps. The first step in data collection is collecting live response. Live response is the process of using an automated tools to collect data both volatile and non-volatile data. After collection live response analysis helps to confirm or deny incidents and help to provide furhter leads, indicators of compromise, hits and compromised hosts. 

## Volatile Data
### Memory
One of the first steps in live response collection is collecting memory. There are several ways to collect memory on Windows systems both command line and GUI tools. For this lab we will be using a command line utility for later automation.

In [1]:
winpmem -file labmemory.img

winpmem : The term 'winpmem' is not recognized as the name of a cmdlet, function, script file, or operable program. 
Check the spelling of the name, or if a path was included, verify that the path is correct and try again.
At line:1 char:1
+ winpmem -file labmemory.img
+ ~~~~~~~
    + CategoryInfo          : ObjectNotFound: (winpmem:String) [], CommandNotFoundException
    + FullyQualifiedErrorId : CommandNotFoundException
 


After collection memory we need to collect data passed on order of volatility. Time is important to every investigation, so collecting date, time and time zone can be done with the command below, go ahead and execute it:

In [2]:
date

Sun Mar  4 15:45:09 Mountain Standard Time 2018


### Network Connections
After the date, time and time zone we will collection network collections, this can be done with netstat on Windows. The `-abn` flag collects all connections and ports (a), associated executable (b), and numerical form of the address and port (n). Go ahead and execute the command below:

In [3]:
netstat -abn


Active Connections

  Proto  Local Address          Foreign Address        State
  TCP    0.0.0.0:135            0.0.0.0:0              LISTENING
  RpcSs
 [svchost.exe]
  TCP    0.0.0.0:445            0.0.0.0:0              LISTENING
 Can not obtain ownership information
  TCP    0.0.0.0:49664          0.0.0.0:0              LISTENING
 Can not obtain ownership information
  TCP    0.0.0.0:49665          0.0.0.0:0              LISTENING
  EventLog
 [svchost.exe]
  TCP    0.0.0.0:49666          0.0.0.0:0              LISTENING
  Schedule
 [svchost.exe]
  TCP    0.0.0.0:49667          0.0.0.0:0              LISTENING
 [spoolsv.exe]
  TCP    0.0.0.0:49668          0.0.0.0:0              LISTENING
 Can not obtain ownership information
  TCP    0.0.0.0:49669          0.0.0.0:0              LISTENING
 [lsass.exe]
  TCP    10.0.2.15:139          0.0.0.0:0              LISTENING
 Can not obtain ownership information
  TCP    10.0.2.15:5040         0.0.0.0:0              LISTENING
  CDPSvc
 [sv

`Proto` is the protocol of the connection, `Local Address` is the IP address and port host we are collecting data from, the `Foreign Address` is the remote IP and port connecting to the host we are collecting from and the `State` is the connection state. We will continue to collect network data by getting DNS cache from the host. DNS cache holds DNS and resolved IP address for the local system. Execute the command below:

In [4]:
ipconfig /displaydns


Windows IP Configuration

    safebrowsing.googleapis.com
    ----------------------------------------
    Record Name . . . . . : safebrowsing.googleapis.com
    Record Type . . . . . : 1
    Time To Live  . . . . : 121
    Data Length . . . . . : 4
    Section . . . . . . . : Answer
    A (Host) Record . . . : 172.217.3.234


    ogs.google.com
    ----------------------------------------
    Record Name . . . . . : ogs.google.com
    Record Type . . . . . : 5
    Time To Live  . . . . : 1
    Data Length . . . . . : 8
    Section . . . . . . . : Answer
    CNAME Record  . . . . : www3.l.google.com


    Record Name . . . . . : www3.l.google.com
    Record Type . . . . . : 1
    Time To Live  . . . . : 1
    Data Length . . . . . : 4
    Section . . . . . . . : Answer
    A (Host) Record . . . : 172.217.3.238


    www.googleapis.com
    ----------------------------------------
    Record Name . . . . . : www.googleapis.com
    Record Type . . . . . : 5
    Time To Live  . . . . : 9

Continuing on the network side we will collect ARP data from the host. ARP will show IP and MAC addresses associated with the system you are collecting data from. ARP data can be collecting from Windows with the command below:

In [5]:
arp -a


Interface: 10.0.2.15 --- 0x2
  Internet Address      Physical Address      Type
  10.0.2.2              52-54-00-12-35-02     dynamic   
  10.0.2.255            ff-ff-ff-ff-ff-ff     static    
  224.0.0.22            01-00-5e-00-00-16     static    
  224.0.0.251           01-00-5e-00-00-fb     static    
  224.0.0.252           01-00-5e-00-00-fc     static    
  239.255.255.250       01-00-5e-7f-ff-fa     static    
  255.255.255.255       ff-ff-ff-ff-ff-ff     static    


In this data you will see common addresses for instance `239.255.255.250` is for the SSDP protocol to discover Plug and Play devices with uPNP. Next we need to collection more netowrk information related to routing and how the system we are collecting routes traffic.

In [6]:
route print

Interface List
  2...08 00 27 70 d2 59 ......Intel(R) PRO/1000 MT Desktop Adapter
  1...........................Software Loopback Interface 1
  4...00 00 00 00 00 00 00 e0 Teredo Tunneling Pseudo-Interface

IPv4 Route Table
Active Routes:
Network Destination        Netmask          Gateway       Interface  Metric
          0.0.0.0          0.0.0.0         10.0.2.2        10.0.2.15     25
         10.0.2.0    255.255.255.0         On-link         10.0.2.15    281
        10.0.2.15  255.255.255.255         On-link         10.0.2.15    281
       10.0.2.255  255.255.255.255         On-link         10.0.2.15    281
        127.0.0.0        255.0.0.0         On-link         127.0.0.1    331
        127.0.0.1  255.255.255.255         On-link         127.0.0.1    331
  127.255.255.255  255.255.255.255         On-link         127.0.0.1    331
        224.0.0.0        240.0.0.0         On-link         127.0.0.1    331
        224.0.0.0        240.0.0.0         On-link         10.0.2.15    281
 

### Processes
Reviewing routes can identify possible compromises in by-passing monitoring points. Next in the order of volatility are processes and related data. First we will collect a process list with Sysinternals `pslist`

In [7]:
pslist


PsList v1.4 - Process information lister
Copyright (C) 2000-2016 Mark Russinovich
Sysinternals - www.sysinternals.com

Process information for DESKTOP-PQ6QQFJ:

Name                Pid Pri Thd  Hnd   Priv        CPU Time    Elapsed Time 
Idle                  0   0   1    0     52     8:33:42.140    39:57:17.927
System                4   8 106 3046    156     0:01:38.171    39:57:17.927
smss                292  11   2   52    464     0:00:00.093    39:57:17.925
csrss               380  13   9  469   1512     0:00:00.703    39:57:00.513
wininit             460  13   1  146   1248     0:00:00.046    39:57:00.266
csrss               472  13  13  459   1756     0:00:04.890    39:57:00.258
winlogon            532  13   4  237   2140     0:00:00.125    39:57:00.220
services            556   9   8  578   4472     0:00:04.953    39:57:00.187
lsass               564   9   9 1197   6020     0:00:08.234    39:57:00.173
fontdrvhost         680   8   5   68   1860     0:00:00.046    39:56:59.975
f

From this output `Name` is the name of the process, `Pid` is the process id, `Pri` is the process priority, `Thd` is the number of threads for the process, `Hnd` is the number of handles, `Priv` is the private virtual memory, `CPU Time` is the time the process has been using the CPU, and `Elapsed Time` is the how long the process has been running. The next collection process point are handles. Handles are objects open by a process. `handles` from Sysinternals will show the process names, process ID, and process owner. Go ahead and execute the command below:

In [8]:
handle


Nthandle v4.11 - Handle viewer
Copyright (C) 1997-2017 Mark Russinovich
Sysinternals - www.sysinternals.com

------------------------------------------------------------------------------
System pid: 4 \<unable to open process>
   9C: File  (---)   \Device\Mup
   C4: File  (---)   \Device\Mup
  100: File  (RW-)   \clfs
  104: File  (RWD)   \clfs
  108: File  (RWD)   \clfs
  10C: File  (RWD)   C:\$Extend\$RmMetadata\$Txf
  110: File  (R--)   \clfs
  114: File  (RWD)   \clfs
  118: File  (R--)   C:\$Extend\$RmMetadata\$TxfLog\$TxfLogContainer00000000000000000003
  11C: File  (R--)   C:\$Extend\$RmMetadata\$TxfLog\$TxfLog.blf
  120: File  (R--)   C:\$Extend\$RmMetadata\$TxfLog\$TxfLogContainer00000000000000000002
  134: File  (R-D)   C:\Windows\System32\drivers\en-US\ntfs.sys.mui
  294: File  (-W-)   C:\pagefile.sys
  37C: File  (R--)   C:\Windows\bootstat.dat
  380: File  (---)   C:\Windows\System32\config\SOFTWARE
  384: File  (---)   C:\Windows\System32\config\SOFTWARE.LOG1
  388: Fil

Next collection point to processes are pipes. Malware will occasioinal use pipes can can be a good indictor of compromose. Execute the command below and review the output

In [9]:
pipelist


PipeList v1.02 - Lists open named pipes
Copyright (C) 2005-2016 Mark Russinovich
Sysinternals - www.sysinternals.com

Pipe Name                                    Instances       Max Instances
---------                                    ---------       -------------
InitShutdown                                      3               -1      
lsass                                             4               -1      
ntsvcs                                            3               -1      
scerpc                                            3               -1      
Winsock2\CatalogChangeListener-318-0              1                1      
epmapper                                          3               -1      
Winsock2\CatalogChangeListener-1cc-0              1                1      
LSM_API_service                                   3               -1      
eventlog                                          3               -1      
Winsock2\CatalogChangeListener-288-0              1     

### Prefetch Files
Other data point for proccess is the prefetch. Prefecht files list the process name and last time the process was executed by reivewing the Last Accessed time stamp and the first time the process executed by reviwing the files creation time. To collect this data you can execute the following PowerShell command:

In [10]:
gci -path C:\windows\prefetch\*.pf -ea 0 | select Name,LastAccessTime,CreationTime | sort LastAccessTime | ConvertTo-CSV

#TYPE Selected.System.IO.FileInfo
"Name","LastAccessTime","CreationTime"
"SVCHOST.EXE-59678011.pf","2/8/2018 8:49:36 PM","2/8/2018 8:49:36 PM"
"WWAHOST.EXE-776591F6.pf","2/8/2018 8:49:36 PM","2/8/2018 8:49:36 PM"
"SMARTSCREEN.EXE-9B5E4173.pf","2/8/2018 8:49:36 PM","2/8/2018 8:49:36 PM"
"RUNTIMEBROKER.EXE-72C0C855.pf","2/8/2018 8:49:36 PM","2/8/2018 8:49:36 PM"
"WMIPRVSE.EXE-1628051C.pf","2/8/2018 8:49:36 PM","2/8/2018 8:49:36 PM"
"DLLHOST.EXE-9037274D.pf","2/8/2018 8:49:38 PM","2/8/2018 8:49:38 PM"
"SPEECHRUNTIME.EXE-22A4873B.pf","2/8/2018 8:50:01 PM","2/8/2018 8:50:01 PM"
"SVCHOST.EXE-678BC29E.pf","2/8/2018 8:50:01 PM","2/8/2018 8:50:01 PM"
"SVCHOST.EXE-FE99AE69.pf","2/8/2018 8:51:06 PM","2/8/2018 8:51:06 PM"
"SVCHOST.EXE-99DDB9AA.pf","2/8/2018 8:51:07 PM","2/8/2018 8:51:07 PM"
"WMIADAP.EXE-F8DFDFA2.pf","2/8/2018 8:53:05 PM","2/8/2018 8:53:05 PM"
"TASKHOSTW.EXE-3E0B74C8.pf","2/8/2018 8:54:06 PM","2/8/2018 8:54:06 PM"
"SVCHOST.EXE-AFDE613F.pf","2/8/2018 8:59:07 PM","2/8/2018 8:59:07 PM

### Logged on Users and Groups
We will move away from processes and look at less volatile data. First we will collect and review loggon sessions. The command `logonsessions` will show logged on users. Execute the command below:

In [11]:
logonsessions -accepteula


LogonSessions v1.4 - Lists logon session information
Copyright (C) 2004-2016 Mark Russinovich
Sysinternals - www.sysinternals.com


[0] Logon session 00000000:000003e7:
    User name:    WORKGROUP\DESKTOP-PQ6QQFJ$
    Auth package: NTLM
    Logon type:   (none)
    Session:      0
    Sid:          S-1-5-18
    Logon time:   3/2/2018 11:48:44 PM
    Logon server: 
    DNS Domain:   
    UPN:          

[1] Logon session 00000000:000057c0:
    User name:    
    Auth package: NTLM
    Logon type:   (none)
    Session:      0
    Sid:          (none)
    Logon time:   3/2/2018 11:48:44 PM
    Logon server: 
    DNS Domain:   
    UPN:          

[2] Logon session 00000000:00005b4b:
    User name:    Font Driver Host\UMFD-1
    Auth package: Negotiate
    Logon type:   Interactive
    Session:      1
    Sid:          S-1-5-96-0-1
    Logon time:   3/2/2018 11:48:44 PM
    Logon server: 
    DNS Domain:   
    UPN:          

[3] Logon session 00000000:00005b75:
    User name:    Font Dr

Next data collection point will be users and groups. The next two command will collect users:

In [12]:
net user


User accounts for \\DESKTOP-PQ6QQFJ

-------------------------------------------------------------------------------
Administrator            DefaultAccount           Guest                    
Topics Test              WDAGUtilityAccount       
The command completed successfully.



The following command shows the groups associated with the system. Execute the command below:

In [13]:
net group

This command can be used only on a Windows Domain Controller.

More help is available by typing NET HELPMSG 3515.



## Non-Volatile Data
### Host Configuration
The next command collects basic host configuration sucha s hostname, OS, IP address and several other basic configuration states. Exectute the following command and review the output.

In [14]:
systeminfo


Host Name:                 DESKTOP-PQ6QQFJ
OS Name:                   Microsoft Windows 10 Enterprise Evaluation
OS Version:                10.0.16299 N/A Build 16299
OS Manufacturer:           Microsoft Corporation
OS Configuration:          Standalone Workstation
OS Build Type:             Multiprocessor Free
Registered Owner:          N/A
Registered Organization:   N/A
Product ID:                00329-20000-00001-AA461
Original Install Date:     2/8/2018, 8:49:03 PM
System Boot Time:          3/4/2018, 12:44:25 AM
System Manufacturer:       innotek GmbH
System Model:              VirtualBox
System Type:               x64-based PC
Processor(s):              1 Processor(s) Installed.
                           [01]: Intel64 Family 6 Model 158 Stepping 9 GenuineIntel ~2808 Mhz
BIOS Version:              innotek GmbH VirtualBox, 12/1/2006
Windows Directory:         C:\WINDOWS
System Directory:          C:\WINDOWS\system32
Boot Device:               \Device\HarddiskVolume1
System Locale

Important data points to note from the `systeminfo` command are as always, Time Zone, OS, Hotfix(s), Domain, and IP Addresses. The next configuration data point to collect is the hosts network configuration for the host, this can be collected with the `ipconfig` command. Execute the command below and review the output. 

In [15]:
ipconfig /all


Windows IP Configuration

   Host Name . . . . . . . . . . . . : DESKTOP-PQ6QQFJ
   Primary Dns Suffix  . . . . . . . : 
   Node Type . . . . . . . . . . . . : Hybrid
   IP Routing Enabled. . . . . . . . : No
   WINS Proxy Enabled. . . . . . . . : No
   DNS Suffix Search List. . . . . . : Home

Ethernet adapter Ethernet:

   Connection-specific DNS Suffix  . : Home
   Description . . . . . . . . . . . : Intel(R) PRO/1000 MT Desktop Adapter
   Physical Address. . . . . . . . . : 08-00-27-70-D2-59
   DHCP Enabled. . . . . . . . . . . : Yes
   Autoconfiguration Enabled . . . . : Yes
   Link-local IPv6 Address . . . . . : fe80::9c46:ba3:eea4:5a7f%2(Preferred) 
   IPv4 Address. . . . . . . . . . . : 10.0.2.15(Preferred) 
   Subnet Mask . . . . . . . . . . . : 255.255.255.0
   Lease Obtained. . . . . . . . . . : Friday, March 2, 2018 11:48:45 PM
   Lease Expires . . . . . . . . . . : Monday, March 5, 2018 12:44:49 PM
   Default Gateway . . . . . . . . . : 10.0.2.2
   DHCP Server . . . . . .

### Scheduled Jobs
Listing of scheduled jobs will show programs scheduled to execute, this can be done with the `schtasks` commad and the Powershell command below, execute it and review the contents:

In [16]:
schtasks


Folder: \
TaskName                                 Next Run Time          Status         
======================================== ====================== ===============
GoogleUpdateTaskMachineCore              3/4/2018 10:31:24 PM   Ready          
GoogleUpdateTaskMachineUA                3/4/2018 4:31:24 PM    Ready          
OneDrive Standalone Update Task-S-1-5-21 3/6/2018 12:13:37 AM   Ready          
User_Feed_Synchronization-{50E0C1B8-CA62 3/4/2018 8:34:42 PM    Ready          

Folder: \Microsoft
TaskName                                 Next Run Time          Status         
======================================== ====================== ===============
INFO: There are no scheduled tasks presently available at your access level.

Folder: \Microsoft\Windows
TaskName                                 Next Run Time          Status         
======================================== ====================== ===============
INFO: There are no scheduled tasks presently available at your a

In [18]:
gwmi -ea 0 Win32_ScheduledJob

### Autoruns and Registry
The next data points are the least volatile of the data points reviewed and collected so far. The next command uses Sysintenrals `autoruns` to gather Windows systems automatic script and executable start locations. The command below stores the autoruns data to an external file with  the `a -autoruns.db` flag and the `-v` confirms the file signature of the identified executables in the autorun location. 

In [19]:
autoruns -a autoruns.db -v

There are more than a several legitimate programs that have autostart locations. You should become familliar with how your enterprise is setup and how each host and program is configured. The next data points will collect data from the Registry, Event Logs and System drives using Windows LogParser. The following command uses LogParser to traverse the registry and store the contents to a seperate file. 

In [20]:
logparser.exe -i:REG -o:CSV "SELECT Path,KeyName,ValueName,Value,LastWriteTime FROM HKLM,HKCU,HKCC,HKCR,HKU" > reg.txt

logparser.exe : The term 'logparser.exe' is not recognized as the name of a cmdlet, function, script file, or operable 
program. Check the spelling of the name, or if a path was included, verify that the path is correct and try again.
At line:1 char:1
+ logparser.exe -i:REG -o:CSV "SELECT Path,KeyName,ValueName,Value,Last ...
+ ~~~~~~~~~~~~~
    + CategoryInfo          : ObjectNotFound: (logparser.exe:String) [], CommandNotFoundException
    + FullyQualifiedErrorId : CommandNotFoundException
 


### Event Logs
The next LogParser command collect Windows event logs into a CSV from the Application, Security and System event logs sources.

In [21]:
logparser.exe -i:EVT -o:CSV "SELECT * FROM System,Application,Security"

logparser.exe : The term 'logparser.exe' is not recognized as the name of a cmdlet, function, script file, or operable 
program. Check the spelling of the name, or if a path was included, verify that the path is correct and try again.
At line:1 char:1
+ logparser.exe -i:EVT -o:CSV "SELECT * FROM System,Application,Securit ...
+ ~~~~~~~~~~~~~
    + CategoryInfo          : ObjectNotFound: (logparser.exe:String) [], CommandNotFoundException
    + FullyQualifiedErrorId : CommandNotFoundException
 


This data can also be collected natively with Powershell Commands on systems Windows 7 and higher. The Cmdlet below will collect the Windows security logs and store them in a CSV file

In [22]:
Get-WinEvent -FilterHashtable @{Logname='security';}  -max 5  | select TimeCreated,ID,Message | ConvertTo-CSV

#TYPE Selected.System.Diagnostics.Eventing.Reader.EventLogRecord
"TimeCreated","Id","Message"
"3/4/2018 3:46:43 PM","4672","Special privileges assigned to new logon.

Subject:
	Security ID:		S-1-5-18
	Account Name:		SYSTEM
	Account Domain:		NT AUTHORITY
	Logon ID:		0x3E7

Privileges:		SeAssignPrimaryTokenPrivilege
			SeTcbPrivilege
			SeSecurityPrivilege
			SeTakeOwnershipPrivilege
			SeLoadDriverPrivilege
			SeBackupPrivilege
			SeRestorePrivilege
			SeDebugPrivilege
			SeAuditPrivilege
			SeSystemEnvironmentPrivilege
			SeImpersonatePrivilege
			SeDelegateSessionUserImpersonatePrivilege"
"3/4/2018 3:46:43 PM","4624","An account was successfully logged on.

Subject:
	Security ID:		S-1-5-18
	Account Name:		DESKTOP-PQ6QQFJ$
	Account Domain:		WORKGROUP
	Logon ID:		0x3E7

Logon Information:
	Logon Type:		5
	Restricted Admin Mode:	-
	Virtual Account:		No
	Elevated Token:		Yes

Impersonation Level:		Impersonation

New Logon:
	Security ID:		S-1-5-18
	Account Name:		SYSTEM
	Account Domain:		N

In this example we filter the output to be only 5 events and you would want to exclude this flag when collecting during a real collection. Additionally you will want to redirect the output of the command to a CSV file instead of the terminal. In addition to the Windows Security event log you would want to collect on the other relavent log sources such as Application and System in the LogName parameter. Examples of these collections are shown in the precending cells, please execute the examples and review the output

In [23]:
Get-WinEvent -FilterHashtable @{Logname='application';}  -max 5  | select TimeCreated,ID,Message | ConvertTo-CSV

#TYPE Selected.System.Diagnostics.Eventing.Reader.EventLogRecord
"TimeCreated","Id","Message"
"3/4/2018 3:46:11 PM","16384","Successfully scheduled Software Protection service for re-start at 2018-04-10T06:18:11Z. Reason: RulesEngine."
"3/4/2018 3:44:00 PM","1","Audio Orchestrator Power Event: Battery Saver Is Not Enabled, Voice Activation Ready"
"3/4/2018 3:42:39 PM","916","MicrosoftEdge (6624,G,0) The beta feature EseDiskFlushConsistency is enabled in ESENT due to the beta site mode settings 0x800000."
"3/4/2018 3:42:24 PM","916","DllHost (5256,G,0) The beta feature EseDiskFlushConsistency is enabled in ESENT due to the beta site mode settings 0x800000."
"3/4/2018 3:42:07 PM","916","MicrosoftEdge (5748,G,0) The beta feature EseDiskFlushConsistency is enabled in ESENT due to the beta site mode settings 0x800000."


In [24]:
Get-WinEvent -FilterHashtable @{Logname='system';}  -max 5  | select TimeCreated,ID,Message | ConvertTo-CSV

#TYPE Selected.System.Diagnostics.Eventing.Reader.EventLogRecord
"TimeCreated","Id","Message"
"3/4/2018 3:42:59 PM","10016","The application-specific permission settings do not grant Local Activation permission for the COM Server application with CLSID 
{D63B10C5-BB46-4990-A94F-E40B9D520160}
 and APPID 
{9CA88EE3-ACB7-47C8-AFC4-AB702511C276}
 to the user DESKTOP-PQ6QQFJ\Topics Test SID (S-1-5-21-414462068-3280499067-173812833-1001) from address LocalHost (Using LRPC) running in the application container Unavailable SID (Unavailable). This security permission can be modified using the Component Services administrative tool."
"3/4/2018 2:13:30 PM","1","The system time has changed to ?2018?-?03?-?04T21:13:30.647000000Z from ?2018?-?03?-?04T09:20:27.609053600Z.

Change Reason: An application or system component changed the time."
"3/4/2018 2:13:30 PM","6013","The system uptime is 48544 seconds."
"3/4/2018 1:30:07 AM","105","Power source change."
"3/4/2018 12:44:31 AM","105","Power source c

## File Listings
In addtion to the event logs you will want to collect some file and folder meta data from the system. The first example of this is using LogParser to create a file listing from the system drive and stores the results to a text file. 

In [25]:
LogParser.exe -i:FS -o:CSV "SELECT * FROM %systemdrive%\*.*" -recurse:-1 > filelisting.txt

LogParser.exe : The term 'LogParser.exe' is not recognized as the name of a cmdlet, function, script file, or operable 
program. Check the spelling of the name, or if a path was included, verify that the path is correct and try again.
At line:1 char:1
+ LogParser.exe -i:FS -o:CSV "SELECT * FROM %systemdrive%\*.*" -recurse ...
+ ~~~~~~~~~~~~~
    + CategoryInfo          : ObjectNotFound: (LogParser.exe:String) [], CommandNotFoundException
    + FullyQualifiedErrorId : CommandNotFoundException
 


You can also execute a similar command with a native Powershell command. The example to list all system drive files is shown in the cell below. Please execute it and review the results

In [26]:
gci -Path $Env:systemdrive\ -force -ea 0 | select fullname,CreationTime,lastwritetime | ConvertTo-CSV 

#TYPE Selected.System.IO.DirectoryInfo
"FullName","CreationTime","LastWriteTime"
"C:\$Recycle.Bin","9/29/2017 7:46:33 AM","2/9/2018 1:14:41 PM"
"C:\Documents and Settings","2/8/2018 7:52:03 PM","2/8/2018 7:52:03 PM"
"C:\iDefense","2/15/2018 7:48:47 PM","2/15/2018 7:48:47 PM"
"C:\PerfLogs","9/29/2017 7:46:33 AM","2/8/2018 11:54:55 PM"
"C:\Program Files","9/29/2017 7:46:33 AM","2/14/2018 7:41:04 PM"
"C:\Program Files (x86)","9/29/2017 7:46:33 AM","2/9/2018 3:43:57 AM"
"C:\ProgramData","9/29/2017 7:46:33 AM","2/9/2018 2:40:06 AM"
"C:\Recovery","2/8/2018 8:48:56 PM","2/8/2018 8:48:56 PM"
"C:\System Volume Information","2/8/2018 7:45:58 PM","2/27/2018 7:47:00 PM"
"C:\tools","2/9/2018 12:13:07 AM","2/9/2018 12:13:07 AM"
"C:\Users","9/29/2017 2:45:11 AM","2/8/2018 9:18:30 PM"
"C:\Windows","9/29/2017 2:45:11 AM","2/15/2018 7:48:47 PM"
"C:\Windows.old","2/8/2018 8:46:25 PM","2/14/2018 11:04:05 PM"
"C:\pagefile.sys","2/8/2018 7:50:03 PM","3/2/2018 11:48:44 PM"
"C:\swapfile.sys","2/8/2018 7:50:03

In this example we only list the files in C: by calling the Windows enviromental variable `$Env:systemdrive\`. When collecting you would want to collect the file listing recursively with the `-r` flag and redirect the output from the console to a file with `> systemdrivefilelisting.csv`